In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import requests

In [ ]:
#for Nifty 500, scraping creates issues
# For Nifty Largemidcap 250 - https://www.niftyindices.com/IndexConstituent/ind_niftylargemidcap250list.csv
# For nifty smallcap 250 - https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap250list.csv

def calculate_price_change(lookback_period):
    stocks_data = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_500')[2]
    stocks_data = (stocks_data[3] + '.NS').to_list() #For Nifty 500. For Nifty fifty can append . Symbol onto stocks data directly
    
    #calclating start and end dates
    end_date = datetime.today() - timedelta(days=1)
    start_date = end_date - timedelta(days = lookback_period)
    
    # Fetch historical data for all stocks
    price_data = yf.download(stocks_data, start=start_date, end=end_date)["Close"]
    
    #calculate price change over the lookback period
    price_change_lookout = (price_data.iloc[-1] / price_data.iloc[0] - 1) * 100
    
    #calculate daily price change and standard deviation
    price_change_daily = price_data.pct_change()
    std_dev_daily = price_change_daily.std()
    
    #calculating the sharpe ratio
    sharpe_df = pd.merge(price_change_lookout.to_frame(), std_dev_daily.to_frame(), on = 'Ticker')
    sharpe_df.columns = ['Return','Std Dev']
    sharpe_df['Sharpe Ratio'] = sharpe_df['Return'] / sharpe_df['Std Dev']

    #printing the top 20 stocks as per the Sharpe ratio
    return sharpe_df.nlargest(20,'Sharpe Ratio')


In [ ]:
#calculate on the basis of 2 years
price_change_yearly = calculate_price_change(730)
price_change_yearly


In [ ]:
#calculate on the basis of 1 year
price_change_yearly = calculate_price_change(365)
price_change_yearly


In [ ]:
#calculate on the basis of 6 months
price_change_6m = calculate_price_change(180)
price_change_6m

In [ ]:
#calculate on the basis of 90 days

price_change_quarterly = calculate_price_change(90)
price_change_quarterly